#1.)set up

In [7]:
#Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import re

I have to sepparate the data from the full data sheet since I want to import it using pandas, which can't read the inturupting headers. To sepparate the data I created a scipt called extractheaders.sh and saved just the end of the headeline numbers in a file that I've loaded bellow to start. I focused on grabbing these numbers because I know they are the start to each data section and I can then count that each header is 13 lines long and I can choose to skip these lines and extract only the data for that section by noting the line the next header starts. 

In [8]:
#load number lines for end of headers from full data 
headerlines=pd.read_table('header_ends.dat')

In [9]:
print(headerlines.head())

     12
0   538
1  1064
2  1590
3  2116
4  2642


In [10]:
#variable for the length of the unseparated data called sabrina.dat
#setting this variable with the len() function would make your code easier to manage if you change anything about your data
#even if you don't change your data, it will make your code more reusable
length_of_file=12624

In [17]:
#this while loop should load your data
#skipped rows is equal to the number of rows at the bottom of the file to skip
skipped_footer = length_of_file
#table will store the each table of data as an item in a list
table = []
#header row is equal to the row that contains the header for the data
header_row = 10
#this loops will continue to go, until all the data has been read
while skipped_footer > 0:
    #shifts the skipped footer down the length of a table 
    skipped_footer -= 526
    #dataframe is the dataframe that contains the data
    dataframe = pd.read_table('sabrina.dat', sep='\s+', header=header_row, skipfooter = skipped_footer, engine='python')
    #appends the dataframe to the list
    table.append(dataframe)
    #save the list of data chunks
    dataframe.to_csv('/Users/sabrinadesoto/Desktop/Sab_Final_Project/Bin/data_chunks_list.csv', delimiter=',')
    #shifts the header row to the next table in the data
    header_row += 523



In [14]:
#here's your data!
#this is table number 4
table[0].head()

#save this table as a file
table[0].to_csv('/Users/sabrinadesoto/Desktop/Data_chunk1.csv', delimiter='\s+')

In [ ]:
#write loop to print separate tables (like above^) for each data section
for row in ('sabrina.dat'):
    table_1=pd.read_table('sabrina.dat', sep='\s+', skiprows=13, skipfooter=length_of_file-527, engine='python')
    
    if row == 'headerlines':
        return(table_1)
    else:
        None
print()
    

In [ ]:
#attempt 2 for writting loop to print separate tables (like above^) for each data section
for row in ('sabrina.dat'):
    if row == 'headerlines':
        table = (pd.read_table('sabrina.dat', sep='\s+', nrows=512, engine='python'))
    print(table)

In [ ]:
#attempt 3 for writting loop to print data 
def print_tables(lines):
    headerlines=pd.read_table('header_ends.dat')
    
    if lines in 'sabrina.dat' & 'headerlines':
        return(pd.read_table('sabrina.dat', sep='\s+', nrows=512))
    
    else:
        None 

In [ ]:
print_tables('sabrina.dat')

#2.)resample data

In [ ]:
#resample data to split it into (30 min ave. chunks)
#necessary for individual plots?-import of pandas and creat new table?

#using reg.ex:
#grab amplitude data
re_ampl=for ampl in readings:
    if re.search('Ampl(Jy) FLAGGED', readings)
    print r

#how do I sepparate for each chunk?

#using iloc:
testload.iloc[:10,2:4] #example the first 10 rows for columns 2 & 3
#how do I know row numbers?

In [ ]:
#grab phase data
re_phase=for phase in readings:
    if re.search('Phase', readings)
    print r

In [ ]:
#creat smaller data set of just phase & amplitude 
resampled=readings[['re_amp','re_phase']]
resampled.head

#add collumn for channel (just counts 1-512)
resampled['Channel']=[]

#3.)make phase diagrams
Spike in the amplitude is the detection we care about. From that we can look at the phase pts that go with that spike.

In [ ]:
#Plot Channel vs. phase/ampl. for all times
#use multi-pannel plots stacked on eachother

#subplot 1-phase
plt.subplot (2,1,1)
#need to limit input to each chunk of data
plt.plot(resampled.Channel, resampled.Phase)
plt.ylabel('Phase')
plt.tittle('Phase Diagram')
#blue line

#subplot 2-ampl.
plt.subplot (2,2,1)
plt.plot(resampled.Channel, resampled.Ampl(Jy))
plt.xlabel('Channel')
plt.ylabel('Amplitude(Jy)')
#green x scatter

In [ ]:
#plot for all chunks of data
resampled.plot(subplots=True, figsize = (10, 12))

#4.)isolate detection data pts.

In [ ]:
#find max amplitude(=detect spike)
print(re_ampl.max())#can I include this in detection function?

def detection_phases(resampled):
'''finds coorisponding phase pts. for amplitude spikes indicating a detection'''
#write loop that for all aplitudes greater than () will return the corresponding phase
    if re_ampl > #value?:
        return re_phase

#loop returns only detection phases 
for data in resampled:
    detection=detection_phases(data)
    if detection:
        print(detection_phases(data))

#5.)compare detection for all 30min ave.

In [ ]:
#creat table from all detection data

#first row is the 30minute data collection chunks

#function to grab times from data: 
def get_time(record):
    '''Return (hour,minute) as string'''
    
    #assign variable to capture pattern and capturing
    cp='\s([\s][0-9]|[0-9][0-9])\s([\s][0-9]|[0-9][0-9])\s30.0'#adjust to include day&ave?
    p=re.search(cp, record)
    
    #loop for research to return above pattern
    if p:
        return p.group(1), p.group(2)

#add loop so that no-matches aren't printed
for r in readings:
    #assign variable to function
    time=get_time(r)
    if time:
        print(get_time(r))
        
#column for each are detection data from step 4
#how do I match those detections with a single detection time for each chunk? 

#6.)do some super handy stats 

#7.)do some way cooler physics